## Downloading Dataset Using Opendatasets

In [1]:
import opendatasets as od

In [2]:
dataset_url = 'https://www.kaggle.com/c/fake-news/data?select=train.csv'

In [3]:
od.download(dataset_url)

Skipping, found downloaded files in ".\fake-news" (use force=True to force download)


In [4]:
import os

In [5]:
data_dir = "./fake-news"

In [6]:
os.listdir(data_dir)

['submit.csv', 'test.csv', 'train.csv']

In [7]:
news_csv = data_dir + '/train.csv'

## Necessary Libraries

In [8]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [9]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [10]:
# prints the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data-Preprocessing

In [11]:
# loading the dataset into a pandas dataframe
data = pd.read_csv(news_csv)

In [12]:
# print the first 5 rows of the dataframe
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
# checks the shape of dataframe
data.shape

(20800, 5)

In [14]:
# gives information about the dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [15]:
# counting the number of missing values in dataframe
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [16]:
# repalcing the null values with empty string
data = data.fillna('')

In [17]:
# again check the null values after replacing with empty string
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [18]:
# merging the author column and title column
data['content'] = data['author']+' '+ data['title']

In [19]:
print(data['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


## Stemming

In [20]:
ps = PorterStemmer()

In [21]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [22]:
data['content'] = data['content'].apply(stemming)

In [23]:
print(data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


## Seprating the data and label

In [24]:
X = data['content'].values
Y = data['label'].values

In [25]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [26]:
print(Y)

[1 0 1 ... 0 1 1]


In [27]:
# converting the textual data into numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [28]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

## train test split

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [30]:
print('Y :', Y.shape)
print('Y_train :', Y_train.shape)
print('Y_test :', Y_test.shape)

Y : (20800,)
Y_train : (16640,)
Y_test : (4160,)


## Model Building

In [31]:
model = LogisticRegression()

In [32]:
model.fit(X_train, Y_train)

LogisticRegression()

In [33]:
X_train_prediction = model.predict(X_train)
print('X_train_prediction :', model.predict(X_train))

X_train_prediction : [0 0 0 ... 0 0 1]


In [34]:
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('training_data_accuracy :', accuracy_score(X_train_prediction, Y_train))

training_data_accuracy : 0.9865985576923076


In [35]:
X_test_prediction = model.predict(X_test)
print('X_test_prediction :', model.predict(X_test))

X_test_prediction : [1 0 1 ... 1 1 0]


In [36]:
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('test_data_accuracy :', accuracy_score(X_test_prediction, Y_test))

test_data_accuracy : 0.9790865384615385


## Losses

In [37]:
mse = mean_squared_error(X_train_prediction, Y_train)
print('mse :', mean_squared_error(X_train_prediction, Y_train))

rmse = np.sqrt(mean_squared_error(X_train_prediction, Y_train))
print('rmse :', np.sqrt(mean_squared_error(X_train_prediction, Y_train)))

mse : 0.013401442307692307
rmse : 0.11576459868065153


In [38]:
mse = mean_squared_error(X_test_prediction, Y_test)
print('mse :', mean_squared_error(X_test_prediction, Y_test))

rmse = np.sqrt(mean_squared_error(X_test_prediction, Y_test))
print('rmse :', np.sqrt(mean_squared_error(X_test_prediction, Y_test)))

mse : 0.020913461538461537
rmse : 0.1446148731578517


## Predictive System

In [39]:
X_new = X_test[15]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print("The news is real")
else:
    print("The news is fake")

[0]
The news is real


In [40]:
print(Y_test[15])

0
